In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from utils.etoile_ventes_avis import graphe_etoile_ventes_avis

DATA_PATH = "../donnees/ecommerce/"

In [ ]:
dot_ventes = graphe_etoile_ventes_avis()
dot_ventes

In [ ]:
fva = pd.read_parquet("f_ventes_avis.parquet").copy()

# Conversion date
if "purchase_timestamp" in fva.columns:
    fva["purchase_timestamp"] = pd.to_datetime(fva["purchase_timestamp"], errors="coerce")
    fva["year"] = fva["purchase_timestamp"].dt.year
    fva["month"] = fva["purchase_timestamp"].dt.month

# Colonnes logistiques attendues en float (heures)
for col in ["approuvee", "envoyee", "livree", "estimee"]:
    if col in fva.columns:
        fva[col] = pd.to_numeric(fva[col], errors="coerce")

# Retard réel (heures) : livrée - estimée
if {"livree", "estimee"} <= set(fva.columns):
    fva["retard_h"] = fva["livree"] - fva["estimee"]

print("Shape fva :", fva.shape)
print("Colonnes :", list(fva.columns))

In [ ]:
# Analyse 2 : Qualité de service logistique & impact sur les avis
# Ici, l’objectif est de relier directement la performance logistique (délais d’approbation,
# d’expédition, de livraison, respect ou non de la date estimée) à la satisfaction client.
# L’idée est de vérifier dans quelle mesure les retards et les délais anormaux se traduisent
# par de mauvais avis (scores 1–2) et des commentaires plus longs ou plus critiques.
# Cette analyse est importante car elle met en évidence des leviers opérationnels concrets
# pour améliorer l’expérience client.

In [ ]:
# Hypothèse 2
# Comme on a observé dans l’analyse 1 que les avis semblaient varier pendant les
# périodes de forte activité, on suppose maintenant que ce comportement est directement
# lié aux délais logistiques. Si les commandes arrivent en retard ou mettent plus de
# temps à être approuvées, le score devrait baisser et les commentaires devraient être
# plus critiques. On s’attend donc à une corrélation claire entre délais, retards et avis.

In [ ]:
# 1 Corrélation délai de livraison réel vs score (version moyenne)

# Préparation
tmp = fva.copy()
tmp = tmp[tmp["livree"].notna() & (tmp["livree"] > 0) & tmp["score"].notna()]
tmp["livree_j"] = tmp["livree"] / 24.0
q99 = tmp["livree_j"].quantile(0.99)
tmp = tmp[tmp["livree_j"] <= q99]

delay_by_score = (
    tmp
    .groupby("score", as_index=False)
    .agg(
        mean_delay_j=("livree_j", "mean"),
        median_delay_j=("livree_j", "median"),
        orders_count=("order_id", "nunique")
    )
)

# Affichage
plt.figure(figsize=(8, 5))
plt.bar(delay_by_score["score"].astype(str), delay_by_score["mean_delay_j"])
plt.xlabel("Score de l'avis")
plt.ylabel("Délai moyen de livraison (jours depuis achat)")
plt.title("Délai moyen de livraison réel par score")
plt.ylim(0, delay_by_score["mean_delay_j"].max() * 1.1)
plt.tight_layout()
plt.show()


In [ ]:
# On observe une relation directe et très nette entre vitesse de livraison et satisfaction.
# Les commandes livrées rapidement obtiennent les meilleures notes, tandis que les délais longs
# tirent systématiquement les scores vers le bas. Les avis à 1 ou 2 étoiles correspondent en grande majorité
# à des livraisons particulièrement lentes, souvent supérieures à deux semaines. À l’inverse, les notes maximales
# sont associées à des délais nettement plus courts. Quelques jours de différence suffisent déjà à modifier la
# perception du client, ce qui montre à quel point la ponctualité logistique pèse dans l’expérience globale.
# La tendance est continue : plus la livraison s’étire, plus la satisfaction diminue, sans rupture ou exception notable.

In [ ]:
# 2 Score moyen selon classe fine de délai de livraison

# Préparation
tmp = fva.copy()
tmp = tmp[tmp["livree"].notna() & (tmp["livree"] > 0) & tmp["score"].notna()]

# Bins de 48h (0–2j, 2–4j, 4–6j, etc.) pour un affichage plus lisible
max_h = tmp["livree"].quantile(0.995)
bins = list(np.arange(0, max_h + 48, 48))
if bins[-1] < tmp["livree"].max():
    bins.append(np.inf)

labels = []
for i in range(len(bins) - 1):
    a, b = bins[i], bins[i + 1]
    if np.isinf(b):
        labels.append(f">{int(a/24)}j")
    else:
        debut = int(a / 24)
        fin = int(b / 24)
        labels.append(f"{debut}–{fin}j")

tmp["delivery_bucket_fine"] = pd.cut(
    tmp["livree"],
    bins=bins,
    labels=labels,
    include_lowest=True,
    right=True
)

delivery_score = (
    tmp
    .groupby("delivery_bucket_fine", observed=False)
    .agg(
        orders_count=("order_id", "nunique"),
        revenue=("value", "sum"),
        avg_score=("score", "mean")
    )
    .reset_index()
    .sort_values("delivery_bucket_fine")
)

# Affichage
plt.figure(figsize=(10, 5))
plt.bar(delivery_score["delivery_bucket_fine"].astype(str), delivery_score["avg_score"])
plt.xlabel("Délai de livraison (classes de 48h)")
plt.ylabel("Score moyen des avis")
plt.title("Score moyen en fonction du délai de livraison (classes fines)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
# Le graphique montre une descente très régulière du score moyen à mesure que le délai de livraison augmente.
# Tant que la commande arrive dans les 4 à 6 jours, la satisfaction reste haute et stable, autour de 4,3 à 4,5.
# Passé le cap des 8 à 10 jours, les notes commencent clairement à baisser, et au-delà de deux semaines,
# la chute devient plus marquée : on tombe rapidement sous les 4, puis sous les 3 autour des 20–25 jours.
# À partir de trois semaines, la tendance est franchement négative : les scores tournent autour de 2,
# voire moins pour les délais les plus extrêmes. On voit donc que quelques jours de plus ne dérangent pas trop,
# mais dès que la livraison s’étire vraiment dans le temps, la perception bascule nettement vers le mécontentement.
# Le comportement observé est très progressif : plus les jours passent, moins les clients sont satisfaits,
# jusqu’à atteindre un niveau d’insatisfaction marqué quand la livraison dépasse un mois.


In [ ]:
# 3 Distribution du retard réel (livrée - estimée)

# Préparation
tmp = fva.copy()
tmp = tmp[tmp["retard_h"].notna()]
tmp["retard_j"] = tmp["retard_h"] / 24.0
tmp = tmp[(tmp["retard_j"] >= -10) & (tmp["retard_j"] <= 10)]

# Affichage
plt.figure(figsize=(8, 5))
plt.hist(tmp["retard_j"], bins=50)
plt.xlabel("Retard réel (jours, livrée - estimée, négatif = livré en avance)")
plt.ylabel("Nombre de commandes")
plt.title("Distribution du retard réel de livraison")
plt.tight_layout()
plt.show()


In [ ]:
# Ce graphique montre que la grande majorité des commandes sont livrées en avance
# par rapport à la date estimée : le retard moyen tourne autour de -4 à -5 jours,
# et la distribution est très concentrée entre -8 et -2 jours. Autrement dit, la
# promesse de livraison est largement conservatrice, ce qui évite la plupart des
# mauvaises surprises côté client. Les vrais retards (valeurs positives) existent,
# mais restent très minoritaires par rapport au volume total. C’est intéressant,
# car malgré leur rareté, nos analyses précédentes montrent qu’ils pèsent fortement
# sur les avis négatifs. Cela suggère qu’un petit nombre de retards génère une
# part disproportionnée des mauvaises notes, ou que le retard n’est qu’un des
# déclencheurs parmi d’autres d’une mauvaise expérience client.


In [ ]:
# 4 Score moyen selon retard réel (catégories)

# Préparation
tmp = fva.copy()
tmp = tmp[tmp["retard_h"].notna() & tmp["score"].notna()]

tmp["retard_cat"] = pd.Series(index=tmp.index, dtype="object")
tmp.loc[tmp["retard_h"] < 0, "retard_cat"] = "livré avant la date"
tmp.loc[(tmp["retard_h"] >= 0) & (tmp["retard_h"] <= 24), "retard_cat"] = "livré à temps (<=1j)"
tmp.loc[(tmp["retard_h"] > 24) & (tmp["retard_h"] <= 24*7), "retard_cat"] = "livré en retard (<=7j)"
tmp.loc[tmp["retard_h"] > 24*7, "retard_cat"] = "retard > 7 jours"

tmp = tmp[tmp["retard_cat"].notna()]

retard_score = (
    tmp.groupby("retard_cat", observed=False)
    .agg(avg_score=("score", "mean"))
    .reset_index()
)

# Affichage
plt.figure(figsize=(8, 5))
plt.bar(retard_score["retard_cat"], retard_score["avg_score"])
plt.xlabel("Catégorie de retard réel")
plt.ylabel("Score moyen des avis")
plt.title("Score moyen selon le retard réel de livraison")
plt.xticks(rotation=20)
plt.tight_layout()
plt.show()


In [ ]:
# Ce graphique montre une rupture très nette dès que la livraison dépasse l’estimé.
# Quand la commande arrive avant la date prévue ou dans la journée attendue, les clients
# restent globalement satisfaits et attribuent des notes supérieures à 4. Dès qu’on bascule
# dans un retard de quelques jours, le score moyen s’effondre autour de 2,8, signe que même
# un léger décalage est très mal vécu. Les retards longs, au-delà d’une semaine, entraînent
# les pires évaluations, avec une moyenne proche de 1,7. On voit donc que l’expérience bascule
# très vite : il suffit de sortir de la fenêtre prévue pour que la perception change
# radicalement. La livraison “dans les temps” n’est donc pas un bonus, mais un prérequis absolu.


In [ ]:
# 5 Taux de retard par mois (heatmap année x mois)

# Préparation
tmp = fva.copy()
tmp = tmp[tmp["retard_h"].notna() & tmp["purchase_timestamp"].notna()]
tmp["is_late"] = tmp["retard_h"] > 0

monthly = (
    tmp
    .groupby(["year", "month"], as_index=False)
    .agg(
        orders_count=("order_id", "nunique"),
        late_orders=("is_late", "sum")
    )
)

monthly = monthly[monthly["orders_count"] > 50]
monthly["late_rate"] = monthly["late_orders"] / monthly["orders_count"]
monthly["late_rate_capped"] = monthly["late_rate"].clip(0, 0.25)

pivot_late = monthly.pivot(index="year", columns="month", values="late_rate_capped")
pivot_late_pct = pivot_late * 100

# Affichage
plt.figure(figsize=(10, 6))
sns.heatmap(
    pivot_late_pct,
    annot=True,
    fmt=".0f",
    cmap="magma",
    vmin=0,
    vmax=25,
    cbar_kws={"label": "Taux de retard (%)"}
)
plt.title("Heatmap du taux de retard de livraison (année x mois)")
plt.xlabel("Mois")
plt.ylabel("Année")
plt.tight_layout()
plt.show()


In [ ]:
# Le graphique met en évidence des variations saisonnières assez nettes dans les retards.
# Les débuts d’année restent globalement maîtrisés, surtout en 2017 où les taux tournent autour de 3 à 6 %.
# En revanche, l’année 2018 montre une montée très forte des retards en février et mars,
# signe probable d’un pic d’activité ou d’un goulot opérationnel à cette période.
# Les mois d’été affichent des taux plus faibles et plus réguliers, ce qui laisse penser
# que les volumes y sont plus faciles à absorber. On observe enfin, en fin d’année 2017,
# un rebond marqué des retards, probablement lié aux opérations de fin d’année.
# L’ensemble suggère que les retards ne sont pas aléatoires mais concentrés sur des
# périodes précises où la chaîne logistique est plus sous tension.```


In [ ]:
# 6 Taux de notes 1–2 selon niveau de retard

# Préparation
tmp = fva.copy()
tmp = tmp[tmp["retard_h"].notna() & tmp["score"].notna()]

conditions = [
    tmp["retard_h"] < 0,
    (tmp["retard_h"] >= 0) & (tmp["retard_h"] <= 24),
    (tmp["retard_h"] > 24) & (tmp["retard_h"] <= 24 * 7),
    (tmp["retard_h"] > 24 * 7),
]
choices = [
    "Livré avant la date",
    "Livré à temps (≤1j)",
    "Retard (1–7j)",
    "Retard (>7j)",
]

tmp["retard_cat"] = np.select(conditions, choices, default="Non classé")
tmp = tmp[tmp["retard_cat"] != "Non classé"]

tmp["is_low_score"] = tmp["score"] <= 2

agg = (
    tmp
    .groupby("retard_cat", observed=False)
    .agg(
        orders_count=("order_id", "nunique"),
        low_reviews=("is_low_score", "sum")
    )
    .reset_index()
)

agg["low_rate"] = agg["low_reviews"] / agg["orders_count"]

cat_order = ["Livré avant la date", "Livré à temps (≤1j)", "Retard (1–7j)", "Retard (>7j)"]
agg["retard_cat"] = pd.Categorical(agg["retard_cat"], cat_order, ordered=True)
agg = agg.sort_values("retard_cat")

print("== 6) Taux de notes 1–2 par niveau de retard ==")
print(agg)

# Affichage
plt.figure(figsize=(8, 5))
plt.bar(agg["retard_cat"], agg["low_rate"])
plt.xlabel("Niveau de retard réel")
plt.ylabel("Part de notes 1–2")
plt.title("Taux de notes 1–2 selon le niveau de retard de livraison")
plt.xticks(rotation=15)
plt.tight_layout()
plt.show()


In [ ]:
# Le graphique montre une montée très nette du risque de notes 1–2 à mesure que le retard augmente.
# Quand la commande arrive avant la date prévue, les avis négatifs restent très rares.
# Livré dans la fenêtre annoncée, le taux grimpe légèrement mais reste encore raisonnable.
# Dès qu’on dépasse la date estimée, la situation change : entre 1 et 7 jours de retard,
# près d’un client sur deux laisse une mauvaise note.
# Au-delà d’une semaine, la rupture est totale : près de 80 % des avis deviennent négatifs.
# On voit donc que le retard agit comme un déclencheur puissant de mécontentement,
# et que chaque jour supplémentaire aggrave fortement la perception du service.

In [ ]:
# 7 Longueur moyenne de commentaire par catégorie de retard

# Préparation
tmp = fva.copy()
tmp = tmp[tmp["retard_h"].notna() & tmp["comment_length"].notna()]

conditions = [
    tmp["retard_h"] < 0,
    (tmp["retard_h"] >= 0) & (tmp["retard_h"] <= 24),
    (tmp["retard_h"] > 24) & (tmp["retard_h"] <= 24 * 7),
    (tmp["retard_h"] > 24 * 7),
]

choices = [
    "livré avant la date",
    "livré à temps (<=1j)",
    "livré en retard (<=7j)",
    "retard > 7 jours",
]

tmp["retard_cat"] = np.select(conditions, choices, default=None)

comment_stats = (
    tmp
    .groupby("retard_cat", observed=False)
    .agg(
        reviews_count=("order_id", "nunique"),
        avg_comment_length=("comment_length", "mean")
    )
    .reset_index()
)

# Affichage
plt.figure(figsize=(8, 5))
plt.bar(comment_stats["retard_cat"], comment_stats["avg_comment_length"])
plt.xlabel("Catégorie de retard réel")
plt.ylabel("Longueur moyenne du commentaire")
plt.title("Longueur moyenne des commentaires selon le retard de livraison")
plt.xticks(rotation=20)
plt.tight_layout()
plt.show()


In [ ]:
# Ce graphique montre que plus la livraison se passe mal, plus les clients prennent le temps d’écrire.
# Quand la commande arrive avant la date prévue ou pile dans la fenêtre annoncée, les commentaires restent
# relativement courts, autour de 28–29 caractères en moyenne. Dès qu’on passe en retard léger (moins d’une semaine),
# la longueur augmente nettement et les messages deviennent plus détaillés, voire plus émotionnels.
# Les retards de plus de 7 jours explosent littéralement la moyenne, avec des commentaires deux fois plus longs que
# pour les livraisons en avance. On sent que les clients utilisent alors l’avis comme un espace pour raconter
# toute leur mauvaise expérience. Globalement, plus le retard est important, plus le client se sent obligé
# d’expliquer, de justifier, voire de “vider son sac” dans le commentaire.

In [ ]:
# 8 Score moyen et taux de notes 1–2 selon délai d’approbation

# Préparation
tmp = fva.copy()
tmp = tmp[tmp["approuvee"].notna() & (tmp["approuvee"] > 0) & tmp["score"].notna()]

bins = [0, 1, 6, 24, 72, np.inf]
labels = ["<=1h", "1–6h", "6–24h", "24–72h", ">72h"]

tmp["approb_bucket"] = pd.cut(
    tmp["approuvee"],
    bins=bins,
    labels=labels,
    include_lowest=True,
    right=True
)

tmp["is_low_score"] = tmp["score"] <= 2

approb_score = (
    tmp
    .groupby("approb_bucket", observed=False)
    .agg(
        orders_count=("order_id", "nunique"),
        avg_score=("score", "mean"),
        low_rate=("is_low_score", "mean")
    )
    .reset_index()
)

approb_score["low_rate_pct"] = approb_score["low_rate"] * 100

# Affichage
fig, axes = plt.subplots(2, 1, figsize=(8, 8), sharex=True)

axes[0].bar(approb_score["approb_bucket"], approb_score["avg_score"])
axes[0].set_ylabel("Score moyen")
axes[0].set_title("Score moyen selon le délai d'approbation")
axes[0].set_ylim(
    approb_score["avg_score"].min() - 0.15,
    approb_score["avg_score"].max() + 0.05
)

axes[1].bar(approb_score["approb_bucket"], approb_score["low_rate_pct"])
axes[1].set_xlabel("Délai d'approbation (heures)")
axes[1].set_ylabel("Taux de notes 1–2 (%)")
axes[1].set_title("Taux de notes 1–2 selon le délai d'approbation")
axes[1].set_ylim(
    approb_score["low_rate_pct"].min() - 1,
    approb_score["low_rate_pct"].max() + 1
)

plt.tight_layout()
plt.show()


In [ ]:
# Les résultats montrent que le délai d’approbation n’a qu’un impact très léger sur la satisfaction.
# Le score moyen reste proche de 4 quelle que soit la catégorie, et seules les approbations
# supérieures à 72 heures affichent une petite baisse visible. Le taux de notes 1–2 reste lui aussi
# contenu, autour de 14 à 17 %, sans vrai décrochage sauf pour les délais les plus longs.
# On voit donc que cette étape n’est pas un irritant majeur pour les clients : tant que la commande
# est validée correctement, quelques heures ou même une journée d’attente ne changent presque rien
# à l’expérience perçue. C’est surtout la suite du parcours logistique qui façonne les avis. ```

In [ ]:
# 9) Score + volume par délai d’expédition (Line + bar)

# Préparation
tmp = fva.copy()
tmp = tmp[tmp["envoyee"].notna() & (tmp["envoyee"] > 0) & tmp["score"].notna()]

bins = [0, 24, 48, 72, 168, np.inf]
labels = ["0–24h", "24–48h", "48–72h", "72–168h", ">168h"]

tmp["exp_bucket"] = pd.cut(tmp["envoyee"], bins=bins, labels=labels, include_lowest=True)

exp_score = (
    tmp
    .groupby("exp_bucket", observed=False)
    .agg(
        avg_score=("score", "mean"),
        volume=("order_id", "nunique")
    )
    .reset_index()
)

# Affichage
fig, ax1 = plt.subplots(figsize=(10, 5))

bars = ax1.bar(exp_score["exp_bucket"], exp_score["volume"], alpha=0.4, label="Volume de commandes")
ax1.set_ylabel("Volume de commandes")

ax2 = ax1.twinx()
(line,) = ax2.plot(exp_score["exp_bucket"], exp_score["avg_score"], marker="o", label="Score moyen")
ax2.set_ylabel("Score moyen")

handles = [bars, line]
labels_legend = [h.get_label() for h in handles]
ax1.legend(handles, labels_legend, loc="upper right")

plt.title("Impact du délai d’expédition : score + volume")
plt.xlabel("Délai d'expédition (heures)")
plt.tight_layout()
plt.show()


In [ ]:
# Ce graphique met bien en évidence le double effet du délai d’expédition : sur la satisfaction,
# mais aussi sur le volume concerné. La majorité des commandes partent dans les 24 à 72 heures,
# avec des scores encore très bons, légèrement au-dessus de 4. Quand l’expédition bascule entre
# 3 et 7 jours, le volume reste élevé mais la note moyenne commence clairement à faiblir.
# La catégorie >168h est la plus problématique : peu de commandes, mais une chute brutale du score,
# signe que ces cas isolés génèrent une forte frustration. On voit donc qu’accélérer l’expédition
# sur les segments lents permettrait à la fois de sécuriser un gros volume et de réduire
# un noyau dur de très mauvaises expériences.


In [ ]:
# Conclusion
# la qualité de la livraison pèse plus que tout sur la satisfaction.
# Quand les commandes arrivent vite et dans la fenêtre annoncée, les clients notent bien et commentent peu.
# Dès que la livraison s’allonge, même de quelques jours, la perception se dégrade et les avis suivent la même pente.
# Les retards dépassant une semaine déclenchent presque mécaniquement des notes très basses et des messages plus longs.
# On constate aussi que certains mois concentrent davantage de tensions logistiques, ce qui amplifie les retards
# et la proportion d’avis négatifs. L’approbation et l’expédition jouent un rôle plus secondaire, mais contribuent
# tout de même à créer des irritants lorsque les délais sortent de l’ordinaire.
# En fin de compte, une minorité de livraisons problématiques produit une grande partie du mécontentement global.
# Le principal levier d’amélioration est donc clair : sécuriser la régularité et la rapidité de la livraison finale.
